In [19]:
!git clone https://github.com/NichoJen/MultiNER.git

fatal: destination path 'MultiNER' already exists and is not an empty directory.


In [20]:
# Insert the directory
import sys
sys.path.insert(0,"/content/MultiNER")

In [21]:
!pip install wandb -qU
!pip install datasets transformers
!pip install evaluate
!pip install seqeval

In [22]:
!unzip models.zip

Archive:  models.zip
replace models/de/xlm_roberta_wiki_neural_de_ep_4.pth? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [23]:
!mkdir results

mkdir: cannot create directory ‘results’: File exists


In [24]:
from datasets import load_dataset
from transformers import XLMRobertaTokenizerFast
from transformers import XLMRobertaModel
import torch
import json
from torch.utils.data import DataLoader
from MultiNER.utils import tokenize_and_align_labels
from MultiNER.XLMRobertaLinearEntityTagger import XLMRobertaLinearEntityTagger
from MultiNER.XLMRobertaLinearEntityTagger import evaluate_model
from pprint import pprint

In [25]:
languages_codes = ["de", "nl", "en", "fr", "it", "es", "pt", "pl", "ru"]

In [26]:
# Hyperparameters
BATCH_SIZE = 64
NUM_CLASSES = 9

In [27]:
# check if gpu available
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {DEVICE}")

Using cuda


In [28]:
# load dataset
wiki_neural_dataset = load_dataset("Babelscape/wikineural")

In [29]:
# load roberta tokenizer
XLMRoberta_tokenizer = XLMRobertaTokenizerFast.from_pretrained("xlm-roberta-base")

In [30]:
# load XLMRoberta model
xlm_roberta_model = XLMRobertaModel.from_pretrained("xlm-roberta-base")

In [31]:
# make dictionary containing all test splits
test_split_dict = dict()
test_loader_dict = dict()
for language in languages_codes:
    print(f"preparing {language} dataset")
    test_set = wiki_neural_dataset["test_" + language]
    print(f"wiki-neural-{language}-test-set: ", test_set)
    #test_set = test_set.select(range(20)) # small set for testing
    test_set = test_set.map(tokenize_and_align_labels, batched=True, batch_size=BATCH_SIZE, fn_kwargs={"tokenizer": XLMRoberta_tokenizer})
    test_set.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
    print("Done!")
    test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, pin_memory=True)
    test_split_dict[language] = test_set
    test_loader_dict[language] = test_loader

preparing de dataset
wiki-neural-de-test-set:  Dataset({
    features: ['tokens', 'ner_tags', 'lang'],
    num_rows: 12372
})


Map:   0%|          | 0/12372 [00:00<?, ? examples/s]

Done!
preparing nl dataset
wiki-neural-nl-test-set:  Dataset({
    features: ['tokens', 'ner_tags', 'lang'],
    num_rows: 10547
})


Map:   0%|          | 0/10547 [00:00<?, ? examples/s]

Done!
preparing en dataset
wiki-neural-en-test-set:  Dataset({
    features: ['tokens', 'ner_tags', 'lang'],
    num_rows: 11597
})


...

Done!
preparing pl dataset
wiki-neural-pl-test-set:  Dataset({
    features: ['tokens', 'ner_tags', 'lang'],
    num_rows: 13585
})


Map:   0%|          | 0/13585 [00:00<?, ? examples/s]

Done!
preparing ru dataset
wiki-neural-ru-test-set:  Dataset({
    features: ['tokens', 'ner_tags', 'lang'],
    num_rows: 11580
})


Map:   0%|          | 0/11580 [00:00<?, ? examples/s]

Done!


In [32]:
models_to_skip = {}

In [33]:
# evaluate models
results_path = "results/"
for language in languages_codes:
    model_path = "models/" + language + "/xlm_roberta_wiki_neural_" + language + "_ep_4.pth"
    print("using model at: " + model_path)
    if language in models_to_skip:
      print("skipped")
      continue
    print("loading model")
    NER_tagger = XLMRobertaLinearEntityTagger(num_classes=NUM_CLASSES, xlm_roberta_model=xlm_roberta_model)
    if torch.cuda.is_available():
      l1_dict = torch.load(model_path)["model_state_dict"]
    else:
      l1_dict = torch.load(model_path, map_location=torch.device('cpu'))["model_state_dict"]
    NER_tagger.l1.load_state_dict(l1_dict)
    NER_tagger.to(DEVICE)
    print("done!")
    results_dict = dict()
    for _language in languages_codes: # evaluate each model on each test set
        print(f"evaluating on {_language} test set")
        test_loader = test_loader_dict[_language]
        results = evaluate_model(NER_tagger, test_loader, device=DEVICE)

        results['accuracy'] = results['accuracy'].item() # accuracy is tensor (not serializable as json file) -> quick fix convert to float
        # number is int 64 (not json serializable by default) -> convert to int
        for entity_type in ['LOC', 'PER', 'ORG', 'MISC']:
          try:
            results['seqeval'][entity_type]['number'] = int(results['seqeval'][entity_type]['number'])
          except KeyError:
            print(f"{entity_type} not in test set")

        pprint(results)
        results_dict[_language] = results

    with open(results_path + "xlm_roberta_results_" + language, "w") as results_file:
        json.dump(results_dict, results_file, indent=4, sort_keys=True)

using model at: models/de/xlm_roberta_wiki_neural_de_ep_4.pth
loading model
done!
evaluating on de test set


100%|██████████| 194/194 [00:45<00:00,  4.27it/s]


{'accuracy': 0.9773256778717041,
 'loss': 0.06968175815706401,
 'seqeval': {'LOC': {'f1': 0.8697169445521521,
                     'number': 5000,
                     'precision': 0.8438675696012039,
                     'recall': 0.8972},
             'MISC': {'f1': 0.5582321321919171,
                      'number': 2638,
                      'precision': 0.5878406708595388,
                      'recall': 0.5314632297194845},
             'ORG': {'f1': 0.840133435976392,
                     'number': 3951,
                     'precision': 0.8519385896435077,
                     'recall': 0.8286509744368514},
             'PER': {'f1': 0.9611609048228766,
                     'number': 5855,
                     'precision': 0.9607508532423208,
                     'recall': 0.9615713065755764},
             'overall_accuracy': 0.9773256680705155,
             'overall_f1': 0.8489439853076216,
             'overall_precision': 0.8499195587221329,
             'overall_recall': 0

100%|██████████| 165/165 [00:41<00:00,  3.98it/s]


{'accuracy': 0.9746530652046204,
 'loss': 0.09966058270497756,
 'seqeval': {'LOC': {'f1': 0.8452705612171666,
                     'number': 6909,
                     'precision': 0.8382918149466192,
                     'recall': 0.8523664785062961},
             'MISC': {'f1': 0.5207990599294947,
                      'number': 2388,
                      'precision': 0.5934654525977504,
                      'recall': 0.4639865996649916},
             'ORG': {'f1': 0.6940326871911819,
                     'number': 1328,
                     'precision': 0.7006907137375288,
                     'recall': 0.6875},
             'PER': {'f1': 0.9377449850126817,
                     'number': 4317,
                     'precision': 0.933440440670186,
                     'recall': 0.9420894139448691},
             'overall_accuracy': 0.9746530374802627,
             'overall_f1': 0.8121651861395539,
             'overall_precision': 0.8230483782297966,
             'overall_recall': 0

...

{'accuracy': 0.965609073638916,
 'loss': 0.12513549710219762,
 'seqeval': {'LOC': {'f1': 0.7454289732770746,
                     'number': 11620,
                     'precision': 0.7384108756227308,
                     'recall': 0.7525817555938038},
             'MISC': {'f1': 0.4384341637010676,
                      'number': 2203,
                      'precision': 0.4592445328031809,
                      'recall': 0.41942805265546984},
             'ORG': {'f1': 0.5774183887391434,
                     'number': 1342,
                     'precision': 0.48272408612919376,
                     'recall': 0.7183308494783904},
             'PER': {'f1': 0.7968538773503748,
                     'number': 4143,
                     'precision': 0.8117175763645468,
                     'recall': 0.7825247405261887},
             'overall_accuracy': 0.9656090662459353,
             'overall_f1': 0.7087398477805589,
             'overall_precision': 0.699133326614935,
             'over

100%|██████████| 213/213 [01:02<00:00,  3.41it/s]


{'accuracy': 0.9616706371307373,
 'loss': 0.135675418544823,
 'seqeval': {'LOC': {'f1': 0.8208013640238705,
                     'number': 11488,
                     'precision': 0.804209822920147,
                     'recall': 0.838091922005571},
             'MISC': {'f1': 0.4374546334381805,
                      'number': 2042,
                      'precision': 0.43232902917264465,
                      'recall': 0.4427032321253673},
             'ORG': {'f1': 0.6586460764948824,
                     'number': 2419,
                     'precision': 0.5822222222222222,
                     'recall': 0.7581645307978504},
             'PER': {'f1': 0.783151073309032,
                     'number': 5969,
                     'precision': 0.7581555834378921,
                     'recall': 0.8098508962975373},
             'overall_accuracy': 0.9616706543617789,
             'overall_f1': 0.7559276594809589,
             'overall_precision': 0.7291534189664675,
             'overall_

100%|██████████| 181/181 [01:02<00:00,  2.92it/s]


{'accuracy': 0.967717170715332,
 'loss': 0.09942382860167251,
 'seqeval': {'LOC': {'f1': 0.864631271747671,
                     'number': 8596,
                     'precision': 0.8352851875948818,
                     'recall': 0.8961144718473709},
             'MISC': {'f1': 0.5767790262172284,
                      'number': 2421,
                      'precision': 0.6655861696380335,
                      'recall': 0.5088806278397356},
             'ORG': {'f1': 0.5971298564928247,
                     'number': 3051,
                     'precision': 0.6406308674427338,
                     'recall': 0.5591609308423467},
             'PER': {'f1': 0.8398133748055988,
                     'number': 3426,
                     'precision': 0.8143679736769948,
                     'recall': 0.8669001751313485},
             'overall_accuracy': 0.9677171548040168,
             'overall_f1': 0.7805143791037074,
             'overall_precision': 0.7830063855490997,
             'overall

In [37]:
!zip -r results.zip results

  adding: results/ (stored 0%)
  adding: results/xlm_roberta_results_en (deflated 78%)
  adding: results/xlm_roberta_results_es (deflated 78%)
  adding: results/xlm_roberta_results_it (deflated 78%)
  adding: results/xlm_roberta_results_ru (deflated 78%)
  adding: results/xlm_roberta_results_de (deflated 78%)
  adding: results/xlm_roberta_results_fr (deflated 79%)
  adding: results/xlm_roberta_results_pt (deflated 78%)
  adding: results/xlm_roberta_results_pl (deflated 78%)
  adding: results/xlm_roberta_results_nl (deflated 78%)


In [38]:
from google.colab import files
files.download("results.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [34]:
results

{'loss': 0.09942382860167251,
 'accuracy': 0.967717170715332,
 'seqeval': {'LOC': {'precision': 0.8352851875948818,
   'recall': 0.8961144718473709,
   'f1': 0.864631271747671,
   'number': 8596},
  'MISC': {'precision': 0.6655861696380335,
   'recall': 0.5088806278397356,
   'f1': 0.5767790262172284,
   'number': 2421},
  'ORG': {'precision': 0.6406308674427338,
   'recall': 0.5591609308423467,
   'f1': 0.5971298564928247,
   'number': 3051},
  'PER': {'precision': 0.8143679736769948,
   'recall': 0.8669001751313485,
   'f1': 0.8398133748055988,
   'number': 3426},
  'overall_precision': 0.7830063855490997,
  'overall_recall': 0.778038184520407,
  'overall_f1': 0.7805143791037074,
  'overall_accuracy': 0.9677171548040168}}